# Higher Order Functions

- Higher Order Functions : function as arguments and return values

# Function as Arguments

## Let's start with our factorial function

For no particular reason. We just need a function that we can time!

In [1]:
l_factorial = lambda n: 1 if n == 0 else n*l_factorial(n-1)

## Timing

### The procedural way, going line by line

Factorial is a recursive and hence time-consuming operation. Let's see how long it takes.

In [2]:
import time

t0 = time.time()
l_factorial(900)
t1 = time.time()
print('Took: %.5f s' % (t1-t0))

Took: 0.00108 s


### The functional way, with a wrapper function

But a better way is to write a wrapper function that times every function that's passed onto it!

In [3]:
def timer(fnc, *arg):
    
    t0 = time.time()
    fnc(*arg)
    t1 = time.time()
    return t1-t0

print('Took: %.5f s' % timer(l_factorial, 900))

Took: 0.00000 s


### The fully functional way, with lambda wrapper functions

We can even turn `timer()` into a lambda function, although it takes a pretty functional state of mind to do so!

In [4]:
l_timestamp = lambda fnc, *arg: (time.time(), fnc(*arg), time.time())
l_diff = lambda t0, retval, t1: t1-t0
l_timer = lambda fnc, arg: l_diff(*l_timestamp(fnc, arg))

print('Took: %.5f s' % l_timer(l_factorial, 900))

Took: 0.00981 s


# Nested Functions

#### Inner and outer functions

Let's start by defining a very basic nested function.

In [5]:
def outer():
    
    def inner():
        
        print('I\'m inner')
    
    inner()

outer()

I'm inner


Now let's all function refer to a variable `x`. This is *the same* variable, the global variable x, in all cases.

In [6]:
def outer():
    
    def inner():
        
        print('Inner:\t\t', x)
    
    print('Outer (before):\t', x)
    inner()
    print('Outer (after):\t', x)

    
x = 'global'
print('Global (before):', x)
outer()
print('Global (after): ', x)

Global (before): global
Outer (before):	 global
Inner:		 global
Outer (after):	 global
Global (after):  global


#### Controlling the variable scope with `global` and `nonlocal`

But as soon as the function assign a new value to `x`, they create their own local variable `x`. So now there are three variables `x`: at the global level, at the level of `outer()`, and at the level of `inner()`. But we can change this using two statements:

  - `global` binds a variable to the global level
  - `nonlocal` (Python >= 3) binds a variable to one level higher

In [7]:
def outer():
    
    def inner():
        
        # nonlocal x
        global x
        x = 'inner'
        print('Inner:\t\t', x)
    
    x = 'outer'
    print('Outer (before):\t', x)
    inner()
    print('Outer (after):\t', x)

    
x = 'global'
print('Global (before):', x)
outer()
print('Global (after): ', x)

Global (before): global
Outer (before):	 outer
Inner:		 inner
Outer (after):	 outer
Global (after):  inner


# Functions as return values

#### Four steps to baking a (pre-baked) croissant

In the weekend, I often eat pre-baked croissants for breakfast. To bake them, you need to perform four steps:

In [8]:
preheat_oven = lambda: print('Preheating oven')
put_croissants_in = lambda: print('Putting croissants in')
wait_five_minutes = lambda: print('Waiting five minutes')
take_croissants_out = lambda: print('Take croissants out (and eat them!)')

Now let's perform all these steps in order!

In [9]:
preheat_oven()
put_croissants_in()
wait_five_minutes()
take_croissants_out()

Preheating oven
Putting croissants in
Waiting five minutes
Take croissants out (and eat them!)


#### Passing all steps to a launcher function

Alternatively, we can create a launcher function (`peform_recipe()`) to which we pass all functions, and which then performs all these functions for us. This is, by itself, not very useful.

In [10]:
def perform_steps(*functions):
    
    for function in functions:
        function()
        
        
perform_steps(preheat_oven,
    put_croissants_in,
    wait_five_minutes,
    take_croissants_out)

Preheating oven
Putting croissants in
Waiting five minutes
Take croissants out (and eat them!)


#### Wrapping all steps into a single recipe

But we can go even further! We can create a `create_recipe()` function that takes all functions, and returns a new function that executes all the passed functions for us!

In [11]:
def create_recipe(*functions):
    
    def run_all():
        
        for function in functions:
            function()
            
    return run_all


recipe = create_recipe(preheat_oven,
    put_croissants_in,
    wait_five_minutes,
    take_croissants_out)
recipe()

Preheating oven
Putting croissants in
Waiting five minutes
Take croissants out (and eat them!)


# The operator module

Let's take our old friend: the factorial function!

In [12]:
l_factorial = lambda n: 1 if n == 0 else n*l_factorial(n-1)
l_factorial(3)

6

#### Chaining functions and combining return values

Say that we want to call this function a number of times, with different arguments, and do something with the return values. How can we do that?

In [13]:
def chain_mul(*what):
    
    """Takes a list of (function, argument) tuples. Calls each
    function with its argument, multiplies up the return values,
    (starting at 1) and returns the total."""
    
    total = 1
    for (fnc, arg) in what:
        total *= fnc(arg)
    return total


chain_mul( (l_factorial, 2), (l_factorial, 3) )

12

#### Operators as regular functions

The function above is not very general: it can only multiple values, not multiply or subtract them. Ideally, we would pass an operator to the function as well. But `*` is syntax and not an object that we can pass! Fortunately, the Python's built-in `operator` module offers all operators as regular functions.

In [14]:
import operator


def chain(how, *what):
        
    total = 1
    for (fnc, arg) in what:
        total = how(total, fnc(arg))
    return total


chain(operator.truediv, (l_factorial, 2), (l_factorial, 3))

0.08333333333333333

In [15]:
chain(operator.mul, (l_factorial, 2), (l_factorial, 3))

12

# Decorators

#### Let's start with our factorial function

For no particular reason. We just need a function that we can time!

In [16]:
def factorial(n):
    
    return 1 if n == 0 else n*factorial(n-1)

#### Another way of timing a function

Here's another way to time a function: by writing a wrapper function (`timer()`) that takes a function as an argument (`fnc`), and returns a new function that times and executes `fnc`!

In [17]:
import time

def timer(fnc):
    
    def inner(arg):
        
        t0 = time.time()
        fnc(arg)
        t1 = time.time()
        return t1-t0
    
    return inner


timed_factorial = timer(factorial)
timed_factorial(500)

0.0004956722259521484

#### That's a decorator! (But there's a nicer syntax)

The `timer` function that we've defined above is a decorator. You can apply a decorator to a function directly, using the `@` syntax.

In [18]:
@timer
def timed_factorial(n):
    
    return 1 if n == 0 else n*factorial(n-1)

timed_factorial(500)

0.0

# Decorator with arguments

#### Let's start with our factorial function

For no particular reason. We just need a function that we can time!

In [19]:
def factorial(n):
    
    return 1 if n == 0 else n*factorial(n-1)

#### What if we want to specify the units of time?

If we want to specify the units of time (seconds or milliseconds), we need to provide the units of time as arguments to the decorator. We can do this, but it requires another level of nesting!

In [20]:
import time


def timer_with_arguments(units='s'):

    def timer(fnc):

        def inner(arg):

            """Inner function"""

            t0 = time.time()
            fnc(arg)
            t1 = time.time()        
            diff = t1-t0
            if units == 'ms':
                diff *= 1000
            return diff

        return inner
    
    return timer


timed_factorial = timer_with_arguments(units='ms')(factorial)
timed_factorial(1000)

61.55848503112793

#### That's a decorator with arguments!


Again, using the `@` syntax, this is gives very clean code!

In [26]:
@timer_with_arguments(units='ms')
def factorial(n):
    
    return 1 if n == 0 else n*factorial(n-1)


factorial(1000)

0.5052089691162109